ch5. RNN
---------

**예제 코드가 전혀 실행되지 않음**
- 굳이 이 책으로 해야 할 필요가 있나 싶다.
- 다른 책으로 해야 겠다

recurrent nueral network(RNN)
- 순환신경망
- 계층의 출력이 순환하는 인공신경망
- 순환 방식은 은닉 계층의 결과가 다음 계층으로 넘어갈 뿐 아니라 자기 계층으로 다시 들어오는 구조
- 따라서 시계열 정보처럼 앞뒤 신호가 서로 상관도가 있는 경우 성능을 더 높일 수 있다.

![alt text](https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile23.uf.tistory.com%2Fimage%2F2578204757AC186F2A70DC)

목차
- RNN의 이해
- 문장을 판별하는 RNN-LSTM
- 시계열 신호열 예측에 RNN-LSTM활용 사례

### 5.1 RNN의 원리

#### 5.1.1 RNN의 개념과 구조

RNN은 신호를 순환하여 시계열 신호와 같이 상호 관계가 있는 신호를 처리하는 구조
- 단순한 방식으로 구현하면 경우에 따라서 학습이 제대로 이뤄지지 않을 수 있음
- 출력된 신호가 계속 순환하면 활성화 함수를 반복적으로 거치게 되어서 gradient를 구하기 어려워짐
  - 학습 방향을 결정하는 연산자인 gradient는 활성화 함수와 연계
  - 신호가 커지면 포화가 되는 특성이 있기 때문에 약간만 입력값이 커져도 미분값이 매우 작아짐
  - 이 때 순환하여 activation function이 반복해서 지나다 보면 미분값이 0에 매우 가까워져 학습이 어려워짐
  - 따라서 이런 문제를 없애려면 곱셉보다는 덧셈을 사용해 과거의 값을 처리하는 것이 유리
- 위의 기본적인 RNN의 문제점을 개선하고자 제안된 것이 LSTM임

#### 5.1.2 LSTM 구조 및 동작

<img src ='image/Keras_RNN_LSTM_structure_example.jpg'>

입력 조절 벡터, 망각 벡터, 출력 조절 벡터를 이용해 입력과 출력 신호를 gating(게이팅) 함
- 게이팅은 신호의 양을 조정해주는 기법
- 입력 조절 벡터는 입력 신호가 tanh 활성화 함수의 완전 연결 계층을 거친 이후의 값을 조절
- 망각 벡터는 과거 입력의 일부를 현재 입력에 반영
- 그리고 출력 조절 벡터는 과거의 값과 수정된 입력값을 고려해 tanh 활성화 함수로 게이팅을 수행
- 게이팅된 최종 결과가 출력 조절 벡터로 나가게 됨
- 이 최종 결과는 데이터 처리를 위한 tanh 계층 그리고 게이팅을 위한 새 시그모이드 계층에 다시 입력으로 들어감

### 5.2 문장을 판별하는 LSTM 구현

LSTM을 이용하여 문장의 의미를 이해 해보는 시간

1. 라이브러리 임포트
2. 데이터 준비
3. 모델링
4. 학습 및 성능 평가

#### 5.2.1 라이브러리 임포트

In [30]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np

from keras.preprocessing import sequence
- pad_sequence()와 같은 sequence를 처리하는 함수를 제공

#### 5.2.2 데이터 준비
- IMDB를 사용
- 영화평과 이진화된 영화 평점 정보(추천=1, 미추천=0)


**코드가 실행이 안 된다** -.-
- https://github.com/jskDr/keraspp/blob/master/ex5_1_lstm_imdb_cl.py

In [34]:
class Data:
    def __init__(self, max_features=20000, maxlen=80):
#         # save np.load
#         np_load_old = np.load

#         # modify the default parameters of np.load
#         np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
        
        (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
        
#         # restore np.load for future normal usage
#         np.load = np_load_old
        
        x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
        x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
        
        # 데이터 셋에 있는 문장들의 길이가 다르므로, LSTM이 처리하기 적합하도록 길이를 통일하는 작업 진행
        # 문장의 최대 길이는 80
        # 문장이 80보다 길이가 작으면 0으로 채움
        # value라는 argu로 값을 채울 수 있음
        
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test

In [35]:
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
#         model = Sequential()
#         model.add(layers.Embedding(max_features, 128))
#         model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
#         model.add(layers.Dense(1, activation='sigmoid'))
        
        # 입력에 각 샘플은 80개의 원소로 된 1차원 신호열이었지만 임베딩 계층을 통과하면서 각 단어가 128의 길이를 가지는 벡터로 바뀌면서 입력 데이터의 모양이 80 x 128로 변경
        x = layers.Input((maxlen, ))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)

        # try using different optimizers and different optimizer configs
        # 긍정인지 부정인지에 대한 이진 판별값을 출력으로 다루므로 loss function은 'binary_crossentropy'
        self.compile(loss='binary_crossentropy',
                     optimizer='adam', metrics=['accuracy'])

In [37]:
class Machine:
    def __init__(self,
                 max_features=20000,
                 maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)

    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model
        print('Training stage')
        print('==============')
        model.fit(data.x_train, data.y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(data.x_test, data.y_test))

        score, acc = model.evaluate(data.x_test, data.y_test,
                                    batch_size=batch_size)
        print('Test performance: accuracy={0}, loss={1}'.format(acc, score))


def main():
    m = Machine()
    m.run()


if __name__ == '__main__':
    main()

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras import models, layers
import seaborn as sns

from kerasapp import skeras


def main():        
    machine = Machine() 
    machine.run(epochs=400)


class Machine():
    def __init__(self):
        self.data = Dataset()
        shape = self.data.X.shape[1:]
        self.model = rnn_model(shape)
        
    def run(self, epochs=400):
        d = self.data
        X_train, X_test, y_train, y_test = d.X_train, d.X_test, d.y_train, d.y_test
        X, y = d.X, d.y
        m = self.model 
        h = m.fit(X_train, y_train, epochs=epochs, validation_data=[X_test, y_test], verbose=0)

        skeras.plot_loss(h)
        plt.title('History of training')
        plt.show()

        yp = m.predict(X_test)
        print('Loss:', m.evaluate(X_test, y_test))
        plt.plot(yp, label='Origial')
        plt.plot(y_test, label='Prediction')
        plt.legend(loc=0)
        plt.title('Validation Results')
        plt.show()

        yp = m.predict(X_test).reshape(-1)
        print('Loss:', m.evaluate(X_test, y_test))  
        print(yp.shape, y_test.shape)

        df = pd.DataFrame()
        df['Sample'] = list(range(len(y_test))) * 2
        df['Normalized #Passengers'] = np.concatenate([y_test, yp], axis=0)
        df['Type'] = ['Original'] * len(y_test) + ['Prediction'] * len(yp)

        plt.figure(figsize=(7, 5))
        sns.barplot(x="Sample", y="Normalized #Passengers", 
                    hue="Type", data=df)
        plt.ylabel('Normalized #Passengers')
        plt.show()
        
        yp = m.predict(X)

        plt.plot(yp, label='Origial')
        plt.plot(y, label='Prediction')
        plt.legend(loc=0)
        plt.title('All Results')
        plt.show()


def rnn_model(shape):
    m_x = layers.Input(shape=shape) #X.shape[1:]
    m_h = layers.LSTM(10)(m_x)
    m_y = layers.Dense(1)(m_h)
    m = models.Model(m_x, m_y)
    
    m.compile('adam', 'mean_squared_error')
    
    m.summary()
    
    return m


class Dataset:
    def __init__(self, fname='international-airline-passengers.csv', D=12):
        data_dn = load_data(fname=fname)
        X, y = get_Xy(data_dn, D=D)
        X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)  
        
        self.X, self.y = X, y
        self.X_train, self.X_test, self.y_train, self.y_test = X_train, X_test, y_train, y_test   


def load_data(fname='international-airline-passengers.csv'):
    dataset = pd.read_csv(fname, usecols=[1], engine='python', skipfooter=3)
    data = dataset.values.reshape(-1)
    plt.plot(data)
    plt.xlabel('Time'); plt.ylabel('#Passengers')
    plt.title('Original Data')
    plt.show()

    # data normalize
    data_dn = (data - np.mean(data)) / np.std(data) / 5
    plt.plot(data_dn)
    plt.xlabel('Time'); plt.ylabel('Normalized #Passengers')
    plt.title('Normalized data by $E[]$ and $5\sigma$')
    plt.show()
    
    return data_dn


def get_Xy(data, D=12):
    # make X and y
    X_l = []
    y_l = []
    N = len(data)
    assert N > D, "N should be larger than D, where N is len(data)"
    for ii in range(N-D-1):
        X_l.append(data[ii:ii+D])
        y_l.append(data[ii+D])
    X = np.array(X_l)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    y = np.array(y_l)
    print(X.shape, y.shape)
    return X, y


if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'international-airline-passengers.csv'